In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import tensorflow as tf
import numpy as np
from IPython.display import Audio
import librosa
# from model.conv_ae import ConvolutionalEncoder, ConvolutionalDecoder, ConvolutionalAutoencoder
# from model.sequence_encoder import SequenceEncoder
# from model.sing import SINGModel

    ## Dataset related functions

In [11]:
# Prepare NSynth TFRecordDataset to pass to model
def prepare_nsynth_dataset(dataset):
    def process_parsed_features(point):
        instrument = tf.one_hot(point['instrument'], 1006)
        pitch = tf.one_hot(point['pitch'], 21)
        velocity = tf.one_hot(point['velocity'], 5)
        audio = point['audio']
        inputs = tf.concat([instrument, pitch, velocity], axis = 0)
        return {'inputs': inputs, 'outputs': audio}

    def parse_nsynth(example_proto):
        features = {
            "audio": tf.io.FixedLenFeature((4 * 16000), tf.float32),
            "note": tf.io.FixedLenFeature((), dtype = tf.int64),
            "note_str": tf.io.FixedLenFeature((), dtype = tf.string),
            "instrument": tf.io.FixedLenFeature((), dtype = tf.int64),
            "instrument_str": tf.io.FixedLenFeature((), dtype = tf.string),
            "instrument_source": tf.io.FixedLenFeature((), dtype = tf.int64),
            "instrument_source_str": tf.io.FixedLenFeature((), dtype = tf.string),
            "instrument_family_str": tf.io.FixedLenFeature((), dtype = tf.string),
            "sample_rate": tf.io.FixedLenFeature((), dtype = tf.int64),
            "velocity": tf.io.FixedLenFeature((), dtype = tf.int64),
            "pitch": tf.io.FixedLenFeature((), dtype = tf.int64),
        }
        parsed_features = tf.io.parse_single_example(example_proto, features)
        return process_parsed_features(parsed_features)

    def tfr_dataset_eager(data, batch_size):
        data = data.apply(tf.data.experimental.shuffle_and_repeat(10000))
        data = data.apply(tf.data.experimental.map_and_batch(map_func = parse_nsynth, batch_size = batch_size))
        data = data.prefetch(1)
        return data

    return tfr_dataset_eager(dataset, 1)

## Data processing utilities

In [12]:
def get_spectrogram_for_audio(raw_wav):
    pass

# Pass in tensor of (64000, ) to get the audio sample in return
def generate_audio_sample(raw_wav):
    np_wav = raw_wav.numpy()
    print(np_wav)
    Audio(np_wav, rate = 16000)
    print('DOne!')

## Create various models that we want to work with

In [ ]:
sing = SING()
conv_autoencoder = ConvolutionalAutoencoder()

## Train the models 

In [13]:
dataset = tf.data.TFRecordDataset('/data/NSynth/nsynth-train.tfrecord')
dataset = prepare_nsynth_dataset(dataset)

for i in dataset.take(1):
    wav = i['outputs']
    generate_audio_sample(wav)

# print('Starting to train the Convolutional Autoencoder ...')
# conv_ae_params = {}
# conv_autoencoder.train(conv_ae_params)
# print('Done!')

# print('Starting to train the SING Model ...')
# sing_params = {}
# sing.train(sing_params)
# print('Done')

[[ 1.0563812e-08 -1.1682036e-08  1.2283434e-08 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]]


## Sample some audio from the train models 

In [ ]:
## TODO - Get some code in here